# Guideline - Longitudinal BMI analysis

## Libraries

```r
# Data wrangling
library(readr)
library(tibble)
library(dplyr, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(purrr)
# Extract estimates from models
library(broom)
library(broom.mixed)
# Fitting models
library(mgcv)
# Stripping models from sensitive data
library(metagam)

## Initial input

The initial input table should include for each individual:

- Sex
- BMI measures with the time when they were collected.

Time is relative to the end of follow-up, which is time = 0. We will estimate trajectories using BMIs prior to the current BMI, which is why time for these measures will be negative.

As an example, here we have a few lines of a table that looks like the one we have generated from the UK Biobank (the data is randomly generated), stored in an object call `input_table`:

```r
input_table <- read_tsv(
  file = "data/input_table.tsv", 
  show_col_types = FALSE
)

head(input_table)

```r
# A tibble: 6 × 5
  eid    sex    bmi0  time   bmi
  <dbl>  <chr>  <dbl> <dbl> <dbl>
1  1     Male    28.5   -2   27.8
2  1     Male    28.5   -1   28.1
3  2     Female  24.3   -2   23.9
4  2     Female  24.3   -1   24.2
5  3     Male    26.7   -2   26.2
6  3     Male    26.7   -1   26.5

## Trajectory model

To model trajectories we will use the linear mixed model framework. We would run this separately for men and women. To do this easily we can work on a ***nested table*** where cells can contain pieces of data (see [here](https://tidyr.tidyverse.org/articles/nest.html)). Here is how to create a nested table separated by sex from our `input_table`:

```r
input_table_bysex <- input_table |>
    nest(LongData = -sex)

```r
print(input_table_bysex)

```r
# A tibble: 2 × 3
  sex    LongData
  <chr>  <list>
1 Male   <tibble [238,036 × 4]>
2 Female <tibble [291,239 × 4]>

Then we can apply the same model to both female and male data using `purrr::map`:

```r
input_table_bysex <- input_table_bysex %>%
  mutate(
    LongMod = map(
      LongData,
      ~lmer(
        formula = bmi ~ bmi0 + time + (time | eid), 
        data = .x, 
        ## Additional options for faster output
        REML = FALSE, 
        control = lmerControl(
          optimizer = "bobyqa", 
          calc.derivs = FALSE
        )
      )
    )
  )


```r
print(input_table_bysex)

```r
# A tibble: 2 × 3
  sex    LongData               LongMod      
  <chr>  <list>                 <list>   
1 Male   <tibble [238,036 × 4]> <lmerMod>
2 Female <tibble [291,239 × 4]> <lmerMod>

## Trajectory estimates

These models contain the estimated average trajectory, but also individual trajectory estimates. We can again use `purrr::map` in the nested table to extract these individual estimates:

```r
input_table_bysex <- input_table_bysex %>%
  mutate(
    # Extracting estimates
    indiv_est = map(LongMod, ~coef(.x)$eid),
    # Putting them in a table
    indiv_est = map(indiv_est, data.frame),
    # Formatting table
    indiv_est = map(
      indiv_est, setNames, 
      nm = c("bmi0", "Slope")
    ),
    # Adding individual Ids - change accordingly
    indiv_est = map(
      indiv_est, 
      rownames_to_column, 
      var = "eid"
    ),
    # Only if Ids are numeric to match data
    indiv_est = map(
      indiv_est, 
      mutate, 
      eid = as.numeric(eid)
    )
  )

print(input_table_bysex)

```r
# A tibble: 2 × 4
  sex    LongData               LongMod       indiv_est        
  <chr>  <list>                 <list>    <list>           
1 Male   <tibble [238,036 × 4]> <lmerMod> <df [57,590 × 3]>
2 Female <tibble [291,239 × 4]> <lmerMod> <df [70,553 × 3]>
```

## Outcomes at end of follow-up

We are interested in assessing the association between the slope and risk biomarkers at the end of follow-up, at the time of the last BMI. Here is how a table of biomarkers should look like:

```r
bmdat <- read_tsv("data/bm_dat.tsv", show_col_types = FALSE)

head(bmdat)

```r
# A tibble: 6 × 22
    eid   sex   age   bmi   ALT   AST   CRP   DBP   GGT
  <dbl> <chr> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
1     1 Male     50  28.5 21.03  22.9  1.39    NA  16.2
2     2 Female   63  24.3 18.41  21.9  1.95  77.5  25.2
3     3 Male     57  26.7 24.67  69.8  2.88  70.0  40.5
4     4 Male     49  33.2 27.97    NA   NA   90.5  45.8
5     5 Male     67  39.8 46.74  40.7  9.41 103.0  81.8
6     6 Male     60  24.2 13.14  24.0  1.20  89.0  22.4
# ℹ 6 more variables: HDL <dbl>, LDL <dbl>, RG <dbl>, 
#   SBP <dbl>, SCR <dbl>, TG <dbl>, WHR <dbl>, 
#   BF_perc <dbl>, AndF_perc <dbl>, GynF_perc <dbl>, 
#   ArmsF_perc <dbl>, LegsF_perc <dbl>, TrunkF_perc <dbl>

We have included in this table the following variables, which are common across the UK Biobank, the Rotterdam Study and ALSPAC. We provide here the units, and we ask to convert your data to these units, so that we can harmonise the results. It is also easier if the variables have the same names as in this table.

|Column type|Biomarker                   |Units                        |Column name  |
|-----------|----------------------------|-----------------------------|-------------|
|Covariate  |Age                         |Years                        |age          |
|Covariate  |BMI                         |km/m2                        |bmi          |
|Covariate  |Sex                         |String ("Female" or "Male")  |sex          |
|Outcome    |Alanine transaminase        |U/L                          |ALT          |
|Outcome    |Aspartate aminotransferase  |U/L                          |AST          |
|Outcome    |C reactive protein          |mg/L                         |CRP          |
|Outcome    |Diastolic blood pressure    |millimeters of mercury (mmHg)|DBP          |
|Outcome    |Fasting glucose             |mmol/L                       |GLU          |
|Outcome    |Gamma glutamyltransferase   |U/L                          |GGT          |
|Outcome    |High density lipoprotein    |mmol/L                       |HDL          |
|Outcome    |Low density lipoprotein     |mmol/L                       |LDL          |
|Outcome    |Serum creatinine            |umol/L                       |SCR          |
|Outcome    |Systolic blood pressure     |millimeters of mercury (mmHg)|SBP          |
|Outcome    |Tryglicerides               |mmol/L                       |TG           |
|Outcome    |Waist-to-hip ratio          |cm/cm                        |WHR          |
|Outcome    |Body fat percentage         |%                            |BF_perc      |
|Outcome    |Trunk fat percentage        |%                            |TrunkF_perc  |
|Outcome    |Arms fat percentage         |%                            |ArmsF_perc   |
|Outcome    |Legs fat percentage         |%                            |LegsF_perc   |
|Outcome    |Gynoid fat percentage       |%                            |GynF_perc    |
|Outcome    |Android fat percentage      |%                            |AndF_perc    |

## Covariates

Here is what we include in UK Biobank, please adjust as needed:
- Current smoking status
- Comorbidities and medication:
  - Diabetes
  - Insulin use
  - Lipid-lowering medication
  - Anti-hypertensive medication
- Socioeconomic status
  - Townsend deprivation index
  - College education
- Sex-specific covariates
  - Menopause
- Ancestry - Genetically inferred

```r
covardat <- read_tsv("data/covardat.tsv", show_col_types = FALSE)

head(covardat)

```r
# A tibble: 6 × 9
  eid   CurrSmoke Diabetes LipidLower HTMed Insulin TSI CollegeEd Menopause ancestry
  <dbl> <dbl>     <dbl>    <dbl>      <dbl> <dbl>   <dbl> <dbl>     <dbl>    <fct>
1     1        0        0          1     0      0   -2.1        1        0    EUR
2     2        1        0          0     1      0    0.5        0        0    EUR
3     3        0        1          1     0      1   -1.3        1        1    EUR
4     4        0        0          0     0      0    1.2        0        0    EUR
5     5        1        1          1     1      0   -0.7        1        1    AFR
6     6        0        0          0     0      0    2.0        0        0    EUR

## Specifying variables

### Age and BMI

```r
agebmi <- c('age', 'bmi')

### Biomarkers

```r
bmrks <- c(
  # Biochemistry markers
  "ALT", "AST", "CRP", "DBP", "GGT", "HDL",
  "LDL", "GLU", "SBP", "SCR", "TG", "WHR",
  # DEXA markers
  "BF_perc", "AndF_perc", "GynF_perc",
  "ArmsF_perc", "LegsF_perc", "TrunkF_perc"
)

### Covariates

```r
covars <- c(
  # Current smoking status
  "CurrSmoke",
  # Comorbidities and medications
  "Diabetes", "LipidLower",
  "HTMed", "Insulin",
  # Socioeconomic covariates
  "TSI", "CollegeEd",
  # Sex-specific covariates
  "Menopause"
  # Ancestry
  "ancestry"
)

## Joining data

```r
joindf <- input_table_bysex %>%
  mutate(
    # Adding biomarkers
    modx = map(indiv_est, inner_join, bmdat),
    # Adding covariates
    modx = map(modx, inner_join, covardat),
    # Selecting columns
    modx = map(
      modx, select, 
      eid, 
      all_of(agebmi),
      Slope,
      any_of(covars),
      any_of(bmrks)
    )
  )

## Reshaping data to run analysis per biomarker

### Pivot longer

```r
joindf <- joindf %>%
  mutate(
    modxl = map(
      modx,
      pivot_longer,
      cols = any_of(bmrks),
      names_to = "trait",
      values_to = "value",
      values_drop_na = TRUE
    )
  )

### Grouping by biomarker

```r
joindf <- joindf %>%
  mutate(
    modxl = map(nest, Data = -trait)
  )

## Fitting models

### Smooth terms

```r
smterms <- c(
  "s(age, bs = \"cr\")",
  "s(bmi, bs = \"cr\")",
  "ti(age, bmi)",
  "s(Slope, bs = \"cr\")",
  "ti(bmi, Slope)"
)

### Covariates

```r
allterms <- c(smterms, covars)

### Running models

```r
joindf <- joindf %>%
  mutate(
    # Within each sex
    modxl = map(
      modxl,
      mutate
      # Within each trait
      mod1 = map(
        Data,
        function(xtrait) {
          mgcv::bam(
            reformulate(
              termlabels = allterms,
              response = "value"
            ),
            data = xtrait
          )
        }
      )
    )
  )

## Removing individual level data from models

Here we use the function from the `metagam` package to strip the model object from any individual level data while retaining all the elements necessary to combine and compare estimates.

```r
joindf <- joindf %>%
  mutate(
    modres = map(
      modxl,
      transmute,
      trait,
      mod1 = map(mod1, strip_rawdata)
    )
  )

## Adding summaries

### Trajectory data

```r
joindf <- joindf %>%
  mutate(
    LongDatSum = map(
      LongData,
      ~.x %>%
          # Extract follow-up time and measures
          group_by(eid) %>%
          summarise(T = min(time), NM = n()) %>%
          # Calculate summary statistics
          summarise(
            TotalInd = n(), TotalT = sum(abs(T)), TotalM = sum(NM),
            across(
              c(T, NM),
              list(
                Mn = mean, Sd = sd, Md = median, 
                Q25 = \(x) quantile(x, probs = .25),
                Q75 = \(x) quantile(x, probs = .75)
              )
            ),
            NM_mode = tibble(X = NM) %>%
              count(X) %>%
              slice_max(n) %>%
              slice_min(as.numeric(X)) %>%
              pull(X),
            NM_modeprop = 100 * sum(NM == NM_mode) / TotalInd
          )
    )
  )

### Trajectory model

```r
joindf <- joindf %>%
  mutate(
    # Average change over time
    LongModSum = map(
      LongMod,
      broom.mixed:::tidy.merMod,
      effects = "fixed",
      conf.int = TRUE
    ),
    # Cleaning
    LongModSum = map(LongModSum, filter, term == "time"),
    LongModSum = map(LongModSum, select, -c(effect, term))
  )

### Regression data

#### Max sample size in regression analyses

```r
joindat <- joindat %>%
  mutate(TotalN = map_dbl(modx, nrow))

#### Summary of continuous variables

```r
# Continous variables, change accordingly
contvars <- c(agebmi, bmrks, "Slope", "TSI")

```r
joindf <- joindf %>%
  mutate(
    bmsum = map(
      modx,
      reframe,
      across(
        any_of(contvars),
        function(x) {
          c(
            quantile(
              x, 
              probs = c(0.025, 0.25, 0.5, 0.75, 0.975), 
              na.rm = TRUE
            ),
            sum(is.na(x))
          )
        }
      )
    )
  )

#### Summary of categorical variables

```r
# Categorical variables, change accordingly
catvars <- covars[covars != 'TSI']

```r
joindf <- joindf %>%
  mutate(
    covarsum = map(
      modx,
      ~.x |>
        count(across(any_of(catvars))) |>
        pivot_longer(
          -n, 
          names_to = "covar_nm", 
          values_to = "covar_val"
        ) |>
        group_by(covar_nm, covar_val) |>
        summarise(nc = sum(n), .groups = "drop")
    )
  )


## Saving trajectory models

We will ask you to save the `input_table_bysex`, which includes the trajectory model and estimates. 

This table might be used for further analyses.

```r
saveRDS(input_table_bysex, "data/input_table_bysex.rds")

## Preparing file for submission

```r
resultdf <- joindf |>
  select(
    sex, 
    LongDatSum,
    LongModSum,
    TotalN,
    bmsum,
    covarsum,
    modres
  )

## Saving submission file

```r
# Cohort - Change accordingly
cohort <- "UKB"

# Saving submission file
saveRDS(
  resultdf
  file = paste0("data/resultdf_", cohort", .rds")
)

## Submit files to cohort folder

Please upload files to the respective cohort folder in the SOPHIA Teams site:

```bash
CrossWP  
└─ "Analysts Working Groups"
    └─ "WG 2"
        └─ BMI_Trajectories_GenPop
           └─ data  
              └─ ***COHORT***

## Thank you!

---

## Session info

```r
R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Red Hat Enterprise Linux 9.4 (Plow)

Matrix products: default
BLAS/LAPACK: /gpfs/gpfs0/Home/daniel_c/miniforge3/envs/RLang/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

time zone: Europe/Stockholm
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] metagam_0.4.0       mgcv_1.9-3          nlme_3.1-168       
 [4] broom.mixed_0.2.9.4 broom_1.0.8         purrr_1.1.0        
 [7] tidyr_1.3.1         dplyr_1.1.4         tibble_3.3.0       
[10] readr_2.1.5        

loaded via a namespace (and not attached):
 [1] Matrix_1.7-3      jsonlite_2.0.0    compiler_4.4.3    crayon_1.5.3     
 [5] tidyselect_1.2.1  IRdisplay_1.1     parallel_4.4.3    splines_4.4.3    
 [9] globals_0.18.0    systemfonts_1.2.3 textshaping_0.3.7 uuid_1.2-0       
[13] fastmap_1.2.0     IRkernel_1.3.2    lattice_0.22-7    R6_2.6.1         
[17] generics_0.1.4    backports_1.5.0   forcats_1.0.0     future_1.58.0    
[21] pillar_1.11.0     tzdb_0.5.0        rlang_1.1.6       repr_1.1.7       
[25] cli_3.6.5         withr_3.0.2       magrittr_2.0.3    grid_4.4.3       
[29] digest_0.6.37     base64enc_0.1-3   hms_1.1.3         pbdZMQ_0.3-11    
[33] lifecycle_1.0.4   vctrs_0.6.5       evaluate_1.0.4    glue_1.8.0       
[37] listenv_0.9.1     furrr_0.3.1       codetools_0.2-20  ragg_1.2.7       
[41] parallelly_1.45.1 tools_4.4.3       pkgconfig_2.0.3   htmltools_0.5.8.1